# 랜덤 포레스트의 개요
* 배깅의 대표적인 알고리즘
* 개별 알고리즘은 결정 트리임.
## RandomForestClassifier 클래스
* 사이킷런의 랜덤 포레스트 클래스
* n_estimator 파라미터로 subset 수 부여

In [2]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(
        data=old_feature_name_df.groupby('column_name').cumcount(),
        columns=['dup_cnt']
    ).reset_index()

    new_feature_name_df = pd.merge(
        old_feature_name_df.reset_index(), feature_dup_df, how='outer'
    )

    # iloc 사용해서 FutureWarning 방지
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(
        lambda x: x.iloc[0] + '_' + str(x.iloc[1]) if x.iloc[1] > 0 else x.iloc[0],
        axis=1
    )

    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [4]:
import pandas as pd

def get_human_dataset( ):
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv(r"C:\Users\HOME\Desktop\human_activity\features.txt",sep=r'\s+',
                        header=None,names=['column_index','column_name'])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame생성. 
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
    X_train = pd.read_csv(r"C:\Users\HOME\Desktop\human_activity\train\X_train.txt",sep=r'\s+', names=feature_name )
    X_test = pd.read_csv(r"C:\Users\HOME\Desktop\human_activity\test\X_test.txt",sep=r'\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv(r"C:\Users\HOME\Desktop\human_activity\train\y_train.txt",sep=r'\s+',header=None,names=['action'])
    y_test = pd.read_csv(r"C:\Users\HOME\Desktop\human_activity\test\y_test.txt",sep=r'\s+',header=None,names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환 
    return X_train, X_test, y_train, y_test


X_train, X_test, y_train, y_test = get_human_dataset()


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 결정 트리에서 사용한 함수를 이용해 학습/테스트용 DF 반환
X_train, X_test, y_train, y_test = get_human_dataset()

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('랜포 정확도: {0:.4f}'.format(accuracy))

랜포 정확도: 0.9237


# 랜덤 포레스트 하이퍼 파라미터 및 튜닝
* n_estimator: 결정 트리의 개수
* max_features, ...

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth': [8, 16, 24],
    'min_samples_leaf': [1, 6, 12],
    'min_samples_split': [2, 8, 16]
}
# RF 객체 생성 후 그리드서치 수행
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(X_train, y_train)

print("최적 파라미터: ", grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

# GBM의 개요
## 부스팅 알고리즘
* weak learner를 순차적 학습/예측하면서 잘못 에측한 데이터에 가중치를 부여하여 오류를 개선해나가며 학습하는 방식
* ex) AdaBoost, 그래디언트 부스트

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print('GBM 수행 시간: {0:.1f}초'.format(time.time()-start_time))

# XGBoost

## XGBoost 개요
* GBM의 단점인 느린 수행 시간 및 과적합 규제 부재 등의 문제를 해결함.
* 파이썬 패키지명: xgboost\